In [1]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.tools import tool
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage

/Users/abhayas/Desktop/machine learning/genAI/langChainENV/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
load_dotenv()

True

In [19]:
llm = HuggingFaceEndpoint(
    repo_id="openai/gpt-oss-20b",
    task ="text-generation",
)
model = ChatHuggingFace(llm= llm)

In [9]:
#creating a tool

@tool 
def multiply(a: int, b: int) -> int:
    """takes 2 numbers a and b and returns the product of that 2 numbers"""
    return a*b

In [10]:
print(multiply.invoke({'a':2,'b':3}))

6


## Tool Binding

In [22]:
model_with_toolBinded = model.bind_tools([multiply])

In [20]:
model.invoke('hello')

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 72, 'total_tokens': 126}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_996f667773', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bebb4-46fd-7700-a16a-35e41fb27e69-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 72, 'output_tokens': 54, 'total_tokens': 126})

In [17]:
model_with_toolBinded.invoke('hello')

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 134, 'total_tokens': 162}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_7d448090ba', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bebb3-c066-7ca3-acea-cad000ec5544-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 134, 'output_tokens': 28, 'total_tokens': 162})

In [23]:
model_with_toolBinded.invoke('multiply 2 and 3')

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiply', 'description': None}, 'id': 'fc_ad437ffd-da51-4bda-8892-875eb0ede0e3', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 139, 'total_tokens': 179}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_334cc21c60', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bebb4-9cd4-75f0-8193-fe18f7c926dd-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': 'fc_ad437ffd-da51-4bda-8892-875eb0ede0e3', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 139, 'output_tokens': 40, 'total_tokens': 179})

in the second case of above there is tool_calls  
model decided it needs tool to solve that even when it could do that multiplication by its own.  
it called the tool and even initialized the arguments required for that tool call

## Tool calling

In [24]:
result = model_with_toolBinded.invoke('multiply 2 and 3')

In [25]:
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiply', 'description': None}, 'id': 'fc_92e5c196-9a76-438c-9522-852a82a53567', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 139, 'total_tokens': 185}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_228717f27c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bebb7-6f1e-7922-9d43-ccb39721d35e-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': 'fc_92e5c196-9a76-438c-9522-852a82a53567', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 139, 'output_tokens': 46, 'total_tokens': 185})

In [27]:
## it is also like a list for multiple tools to use in tool_calls
result.tool_calls

[{'name': 'multiply',
  'args': {'a': 2, 'b': 3},
  'id': 'fc_92e5c196-9a76-438c-9522-852a82a53567',
  'type': 'tool_call'}]

In [28]:
result.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 2, 'b': 3},
 'id': 'fc_92e5c196-9a76-438c-9522-852a82a53567',
 'type': 'tool_call'}

## Tools executing

in the second when we give all the outputs of tool_calls[0]  
it returns in the format of ToolMessage good for storing like HumanMessage or AIMessage while sending multiple messages

In [29]:
multiply.invoke(result.tool_calls[0]['args'])

6

In [30]:
multiply.invoke(result.tool_calls[0])

ToolMessage(content='6', name='multiply', tool_call_id='fc_92e5c196-9a76-438c-9522-852a82a53567')